In [134]:
import folium
from folium import plugins
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

In [135]:
# Coordinates for tornado disasters in Columbus, 1950-2010: http://www.usa.com/columbus-oh-natural-disasters-extremes.htm
tornado_df = pd.read_csv("HistoricalTornadoEvents.csv")
tornado_df.dropna(axis=0, inplace=True)
tornado_df

,Distance (miles),Date,Magnitude,Start Lat/Log,End Lat/Log,Length,Width,Fatalities,Injuries,Property Damage,Crop Damage,Affected County
0,5.2,5/10/1973,3,39°55'N / 83°15'W,39°55'N / 82°48'W,23.80 Miles,300 Yards,0.0,3.0,3K,0,Franklin
1,5.5,2/22/1971,3,39°58'N / 82°56'W,40°02'N / 82°50'W,6.80 Miles,100 Yards,0.0,7.0,2.5M,0,Franklin
5,12.6,4/3/1974,2,40°05'N / 82°51'W,40°08'N / 82°46'W,5.10 Miles,90 Yards,0.0,0.0,250K,0,Franklin
6,12.6,10/11/2006,2,40°05'N / 82°47'W,40°05'N / 82°47'W,1.00 Mile,150 Yards,0.0,0.0,50.0M,0K,Franklin
8,13.1,5/8/1973,2,40°06'N / 83°07'W,40°15'N / 82°51'W,17.30 Miles,67 Yards,0.0,0.0,250K,0,Franklin
12,15.2,9/20/2000,2,40°08'N / 82°50'W,40°11'N / 82°47'W,5.00 Miles,100 Yards,0.0,2.0,1.0M,0,Delaware
15,18.3,5/10/1973,3,39°55'N / 82°48'W,39°55'N / 82°30'W,15.80 Miles,300 Yards,0.0,0.0,3K,0,Fairfield
16,21.0,3/10/1986,2,39°43'N / 83°12'W,39°45'N / 83°13'W,3.00 Miles,100 Yards,0.0,10.0,2.5M,0,Pickaway
17,21.6,5/10/1973,3,39°55'N / 83°31'W,39°55'N / 83°15'W,14.10 Miles,300 Yards,0.0,0.0,3K,0,Madison
18,22.7,9/3/1970,3,39°39'N / 82°58'W,39°40'N / 82°55'W,2.30 Miles,100 Yards,0.0,1.0,25K,0,Pickaway


In [136]:
def deg_to_decimal(d, m, s, direction):
    res = int(d) + float(m)/60 + float(s)/3600
    if direction in ('S','W'):
        res *= -1
    return res

def tornado_df_coords(str_col):
    tornado_arr = np.array(tornado_df[str_col])
    coords = [coord.split(" / ") for coord in tornado_arr]
    print(coords)

    p = re.compile('(\d+)°(\d+)\'([N|S|E|W])')
    coords_new = []
    for lat_long in coords:
        lat = p.match(lat_long[0])
        lat_dd = deg_to_decimal(lat.group(1), lat.group(2), 0, lat.group(3))

        long = p.match(lat_long[1])
        long_dd = deg_to_decimal(long.group(1), long.group(2), 0, long.group(3))
        
        coords_new.append([lat_dd, long_dd])

    return coords_new

starts = tornado_df_coords('Start Lat/Log')
print(starts)
ends = tornado_df_coords('Start Lat/Log')
print(ends)

[["39°55'N", "83°15'W"], ["39°58'N", "82°56'W"], ["40°05'N", "82°51'W"], ["40°05'N", "82°47'W"], ["40°06'N", "83°07'W"], ["40°08'N", "82°50'W"], ["39°55'N", "82°48'W"], ["39°43'N", "83°12'W"], ["39°55'N", "83°31'W"], ["39°39'N", "82°58'W"], ["39°43'N", "83°16'W"], ["39°50'N", "83°34'W"], ["40°11'N", "82°44'W"], ["39°36'N", "82°59'W"], ["39°35'N", "82°58'W"], ["40°23'N", "83°15'W"], ["40°01'N", "83°36'W"], ["39°55'N", "82°30'W"], ["40°02'N", "82°27'W"], ["40°25'N", "83°01'W"], ["40°10'N", "82°43'W"], ["40°05'N", "82°25'W"], ["40°14'N", "82°29'W"], ["39°29'N", "83°34'W"], ["40°30'N", "83°11'W"], ["39°47'N", "83°43'W"], ["40°30'N", "82°54'W"], ["39°45'N", "82°24'W"], ["40°26'N", "82°56'W"], ["40°21'N", "83°33'W"], ["40°30'N", "82°45'W"], ["40°31'N", "83°12'W"], ["40°23'N", "83°32'W"], ["40°14'N", "83°50'W"], ["39°30'N", "83°29'W"], ["39°29'N", "83°34'W"], ["40°06'N", "83°50'W"], ["40°04'N", "82°15'W"], ["40°36'N", "82°40'W"], ["40°14'N", "82°11'W"], ["40°40'N", "82°54'W"], ["40°36'N", "82

In [165]:
def tornado_map(coords):
    tornado_start_map = folium.Map([39.9612, -82.9988], zoom_start=8)
    for i in range(len(coords)):
        magnitude = float(list(tornado_df['Magnitude'])[i])
#         dist = float(list(tornado_df['Distance (miles)'])[i])
        tornado_start_map.add_child(plugins.HeatMap([coords[i]], radius=magnitude*5))
    return tornado_start_map
    
tornado_map(starts)

In [166]:
tornado_map(ends)

### Insights
The above maps are plots of the starting and ending coordinates of 46 different tornadoes of magnitude 2 or above that occurred between 1950 and 2010. In that time period, only two tornadoes started and ended within the city perimeters of Columbus, Ohio. One of the tornadoes was a magnitude 2 and the other a magnitude 3. Tornado events are unlikely to pose a major risk to electric vehicle charging stations built in Columbus.